In [33]:
import re

haveNumberExecptZero=' adj([1-9]\d*) '
haveOnlyZero='( adj[0]\d* )'
onlyAdj = '( adj )'
onlySpace = '( )'


zeroDistance = r'|'.join((haveOnlyZero, onlyAdj, onlySpace))

line = '조성물 청구 항' 
# line = '조성물 adj 청구 adj 항'
# line = '조성물 adj 청구 ADJ 항'
# line = '조성물 adj1 청구 adj3 항'
# line = '조성물 adj0 청구 adj10 항'

# line = re.sub(zeroDistance, '<->', line, flags=re.IGNORECASE)

line = re.sub(haveNumberExecptZero, r"<\1>", line, flags=re.IGNORECASE)
line = re.sub(zeroDistance, r"<->", line, flags=re.IGNORECASE)
# line = re.sub(reAdjOnlyZero, r"<->", line, flags=re.IGNORECASE)
# line = re.sub(reOnlyAdj, "<->", line, flags=re.IGNORECASE)
# line = re.sub(reOnlySpace, "<->", line, flags=re.IGNORECASE)
print(line)    

조성물<->청구<->항


In [90]:
import re
def tsquery_keywords():
    def changeNear(v):
        haveNumberExecptZero=' near([1-9]\d*) '
        haveOnlyZero='( near[0]\d* )'
        onlyAdj = '( near )'
        findDelimiter = '(<[\d+|-]>)'
        # onlySpace = '( )'
        zeroDistance = r'|'.join((haveOnlyZero, onlyAdj, onlySpace))
        v = re.sub(haveNumberExecptZero, r"<\1>", v, flags=re.IGNORECASE)
        v = re.sub(zeroDistance, r"<->", v, flags=re.IGNORECASE)
        delimiter = re.search(findDelimiter, v, flags=re.IGNORECASE).group(1)
        return v, delimiter

    def swapPositions(val): 
        foo = val.partition(delimiter)
        return foo[2] + foo[1] + foo[0]

    # line = 'A or B near C'
    # line = 'A near B or C'
    line = 'A or B near C or D near E or F'
    # line = '조성물 청구 항' 
    # line = '조성물 near 청구 near 항'
    # line = '조성물 adj 청구 ADJ 항'
    # line = '조성물 adj1 청구 adj3 항'
    # line = '조성물 adj0 청구 adj10 항'
    strNear = ""
    strKeyword = ""
    # line = re.sub(zeroDistance, '<->', line, flags=re.IGNORECASE)
    if ' near' in line.lower():
        for v in re.split(" or ", line, flags=re.IGNORECASE):
            v = re.sub('[()]', '', v)
            if ' near' in v.lower():
                val, delimiter = changeNear(v)
                bar = swapPositions(val)
                strNear += "(" + val + "|" + bar + ")|"
            else:
                strNear += "".join(str(v)) + "|"

        if strNear.endswith("|"):
            strNear = strNear[:-1]

        strKeyword += strNear
        val = ""
    return strKeyword        


# line = re.sub(haveNumberExecptZero, r"<\1>", line, flags=re.IGNORECASE)
# line = re.sub(zeroDistance, r"<->", line, flags=re.IGNORECASE)
# delimiter = re.search(findDelimiter, line, flags=re.IGNORECASE).group(1)
# line = re.sub(reAdjOnlyZero, r"<->", line, flags=re.IGNORECASE)
# line = re.sub(reOnlyAdj, "<->", line, flags=re.IGNORECASE)
# line = re.sub(reOnlySpace, "<->", line, flags=re.IGNORECASE)
print(tsquery_keywords()) 

A|(B<->C|C<->B)|(D<->E|E<->D)|F


In [57]:
import re
def tsquery_keywords():

    adjHaveNumberExecptZero=' adj([1-9]\d*) '
    adjHaveOnlyZero='( adj[0]\d* )'
    adjOnly = '( adj )'
    onlySpace = '( )'
    adjZeroGroup = r'|'.join((adjHaveOnlyZero, adjOnly, onlySpace))
    adjSpace = '(?<!or)(\s)(?!or)'

    nearHaveNumberExecptZero=' near([1-9]\d*) '
    nearHaveOnlyZero='( near[0]\d* )'
    nearOnly = '( near )'
    findDelimiter = '(<[\d+|-]>)'
    nearZeroGroup = r'|'.join((nearHaveOnlyZero, nearOnly))

    def changeAdj(v):
        v = re.sub(adjHaveNumberExecptZero, r"<\1>", v, flags=re.IGNORECASE)
        v = re.sub(adjZeroGroup, r"<->", v, flags=re.IGNORECASE)
        return v

    def changeNear(v):

        def swapPositions(val, delimiter): 
            foo = val.partition(delimiter)
            return foo[2] + foo[1] + foo[0]    

        v = re.sub(nearHaveNumberExecptZero, r"<\1>", v, flags=re.IGNORECASE)
        v = re.sub(nearZeroGroup, r"<->", v, flags=re.IGNORECASE)
        delimiter = re.search(findDelimiter, v, flags=re.IGNORECASE).group(1)
        bar = swapPositions(v, delimiter)
        return "(" + v + "|" + bar + ")"

    def changeOr(v):
        v = re.sub(' or ', r"|", v, flags=re.IGNORECASE)
        return v + ")"

    def cutDelimiter(foo, bar):
        if foo.endswith(foo):
            bar = -len(bar)
            foo = foo[:bar]
        return foo  

    keyword = "예방 or 치료 adj 질병 and 진단 and 진단 키트 or 뭥미"
    print(keyword)

    if keyword and keyword != "":
        strKeyword = ""
        for val in re.split(" and ", keyword, flags=re.IGNORECASE):    
            # convert nagative - to !
            if val.startswith("-") or ' or -' in val:
                val = val.replace("-", "!")
            # convert nagative not to !
            if val.startswith("not ") or ' or not ' in val:
                val = val.replace("not ", "!")
            # convert wildcard * to :*
            if val.endswith("*") or '*' in val:
                val = val.replace("*", ":*")
            
            for v in re.split(" or ", val, flags=re.IGNORECASE):
                strKeyword += "("

                strAdj = ""
                if ' adj' in val.lower():
                    v = re.sub('[()]', '', val)
                    strAdj += changeAdj(v)
                elif ' near' in v.lower():
                    strAdj += changeNear(v)
                elif re.search(adjSpace, val, flags=re.IGNORECASE):                   
                    strAdj += changeAdj(v)
                else:                        
                    strAdj += "".join(str(v))
                strKeyword += strAdj + ")|"               
                val = ""

                # strNear = ""
                # if ' near' in val.lower():
                #     v = re.sub('[()]', '', val)
                #     strNear += changeNear(v)
                # elif ' adj' in v.lower():
                #     strNear += changeAdj(v)
                # else:                        
                #     strNear += "".join(str(v))
                # strKeyword += strNear + ")"
                # val = ""

                # strAdj = ""
                # if re.search(adjSpace, val, flags=re.IGNORECASE): # A B
                #     v = re.sub('[()]', '', val)
                #     strAdj += changeAdj(v)
                #         else:                        
                #             strAdj += ")|"+"".join(str(v)) + "|"
                #     strKeyword += cutDelimiter(strAdj, "|")
                #     val = ""
                #     print(strKeyword)
                # # if " or ".upper() in map(str.upper, val):
                # #     needPlainto = "\""                

                strKeyword += (
                    "".join(str(val))
                )          
        # strKeyword = strKeyword.replace(" AND ", needPlainto + " & ").replace(" OR ", needPlainto + " | ").replace(" and ", needPlainto + " & ").replace(" or ", needPlainto + " | ")
        strKeyword = cutDelimiter(strKeyword, "&")              
        if not strKeyword:
            return None

    return strKeyword        

print(tsquery_keywords())     

# 진단 키드 <-> 진단 or 키트 

예방 or 치료 adj 질병 and 진단 and 진단 키트 or 뭥미
(예방<->or<->치료<->질병)|(치료 adj 질병)|(진단)|(진단<->키트)|(뭥미)


In [104]:
# keyword = '예방 or 치료 adj 질병 or 진단 NEAR2 방어 or 치료 and 진단 키트 or 뭥미 and 진단 NEAR0 방어'
# keyword = '예방 and 치료 and 질병 and (진단 키트)'
# keyword = '예방 치료 질병 (진단 키트)'
keyword = '예방 or 치료 adj 질병 or 진단 NEAR2 방어 or 치료 and 진단 키트 or 뭥미 and 진단 NEAR0 방어'


print(keyword)

adjHaveOnlyZero='( adj[0]\d* )'
adjOnly = '( adj )'
adjZeroGroup = r'|'.join((adjHaveOnlyZero,adjOnly))
adjHaveNumberExecptZero=' adj([1-9]\d*) '
adjSpace = r'(\([-!:*ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) ([-!:*ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+\))'
# nearHaveOnlyZero='( near[0]\d* )'
# nearOnly = '( near )'
nearZeroGroup = r'|'.join((nearHaveOnlyZero,nearOnly))
# nearHaveNumberExecptZero=' near([1-9]\d*) '


nearHaveNumberExecptZero = '([-|!|:|*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) near([1-9]\d*) ([^\)\&]+)'
nearHaveOnlyZero='([-|!|:|*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) near[0]\d* ([^\)\&]+)'
nearOnly='([-|!|:|*|ㄱ-ㅎ|가-힣|a-z|A-Z|0-9]+) near ([^\)\&]+)'
# adjSpace = '(?<!or)(\s)(?!or)'
onlySpace = '( )'
# removeDuplicateBrackets = '~[\(\)]+~'


findDelimiter = '(<[\d+|-]>)'
nearZeroGroup = r'|'.join((nearHaveOnlyZero, nearOnly))

existOr = True if ' or ' in keyword else False
strKeyword = "(" if existOr else ''
v = keyword
v = re.sub(' or ', r")|(", v, flags=re.IGNORECASE)
v = re.sub(adjZeroGroup, r"<->", v, flags=re.IGNORECASE)
v = re.sub(adjHaveNumberExecptZero, r"<\1>", v, flags=re.IGNORECASE)
v = re.sub(' and ', r"&", v, flags=re.IGNORECASE)
v = re.sub(nearHaveNumberExecptZero, r"(\1<\2>\3|\3<\2>\1)", v, flags=re.IGNORECASE)
# v = re.sub(removeDuplicateBrackets, r"", v, flags=re.IGNORECASE)
v = re.sub(nearZeroGroup, r"(\1<->\2|\2<->\1)", v, flags=re.IGNORECASE)
v = re.sub(adjSpace, r"\1<->\2", v, flags=re.IGNORECASE)
v = re.sub(' ', r"&", v, flags=re.IGNORECASE)

strKeyword += v + ")" if existOr else v
print(strKeyword)

예방 치료 질병 (진단 키트)
예방&치료&질병&(진단<->키트)
